1. LLM獲取文字
2. 判斷人物、說話語氣
3. TTS轉語音

In [3]:
import gc
import re
import json
import pandas as pd
from pydub import AudioSegment

from TTS.api import TTS
from gliner import GLiNER
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

from utils import *

device = "cuda" if torch.cuda.is_available() else "cpu"

cpu


In [6]:
gn = GLiNER.from_pretrained("urchade/gliner_mediumv2.1").to(device)
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)
classifier = pipeline("sentiment-analysis", model="michellejieli/emotion_text_classifier",  device=device)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


 > tts_models/multilingual/multi-dataset/xtts_v2 has been updated, clearing model cache...
 > Downloading model to /Users/wangqiqian/Library/Application Support/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|████████████████████████████████████████████▉| 1.87G/1.87G [08:00<00:00, 4.30MiB/s]
100%|█████████████████████████████████████████████| 1.87G/1.87G [08:01<00:00, 3.88MiB/s]
100%|█████████████████████████████████████████████| 4.37k/4.37k [00:00<00:00, 9.91kiB/s]

100%|████████████████████████████████████████████████| 361k/361k [00:00<00:00, 718kiB/s]
100%|███████████████████████████████████████████████| 32.0/32.0 [00:00<00:00, 25.4kiB/s]


 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


GPT2InferenceModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed 

In [36]:
# with open('fairy_tales/1.txt', 'r', encoding='utf-8') as file:  # 使用 'utf-8' 確保支援中文或其他特殊字元
#     content = file.read()

content = """
"I don't think we should go any further," John said nervously.
"Come on, don't be such a coward," Sarah replied with a smirk.
The forest was dark, and the wind howled through the trees.
"But what if we get lost?" John asked, his voice trembling.
"We'll be fine," Sarah reassured him. "I've been here before."
They continued walking, the sound of crunching leaves underfoot the only noise accompanying them.
"""

In [ ]:
for d in range(df.shape[0]):
    
    tts.tts_to_file(
        text=df['content'][d],
        # speaker = df['char'][d],
        speaker_wav="Data/03-01-01-01-01-01-06.wav" if df['char'][d],
        language="en",
        emotion = df['emotion'][d],
        file_path=f"Results/output{d}.wav")

In [52]:
result = llm(content)

df = pd.DataFrame(result)
df['emotion'] = None
df['char'] = None

labels = ["Male", "Female", "Other"]

# 獲取情緒和角色身分
for c in range(df.shape[0]):
    sentence_with_name = f"{df['speaker'][c]} : {df['content'][c]}"
    sentence_only = df['content'][c]
    
    classify_result = classifier(sentence_only)
    gn_result = gn.predict_entities(sentence_with_name, labels, threshold=0.5)
    
    df.loc[c, 'emotion'] = classify_result[0]['label'] 
    df.loc[c, 'char'] = gn_result[0]['label']

speaker_mapping = {
    'Female': "Data/03-01-01-01-01-01-06.wav",
    'Male': "Data/03-01-01-01-01-01-05.wav"
}
default_speaker = "Data/03-01-01-01-01-01-11.wav"

# 文字到語音
for d in range(df.shape[0]):
    speaker_wav = speaker_mapping.get(df['char'][d], default_speaker)
    
    tts.tts_to_file(
        text=df['content'][d],
        speaker_wav=speaker_wav,
        language="en",
        emotion=df['emotion'][d],
        file_path=f"Results/output{d}.wav"
    )

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ./Model/mistral-7b-instruct-v0.1.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llam

模型載入成功！
開始分析文本...

找到 10 段對話:


llama_perf_context_print:        load time =   31969.51 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   365 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   236 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   77440.88 ms /   601 tokens



模型原始輸出:
[
    {
        "speaker": "John",
        "content": "I don't think we should go any further, John said nervously."
    },
    {
        "speaker": "Sarah",
        "content": "Come on, don't be such a coward, Sarah replied with a smirk."
    },
    {
        "speaker": "The wind",
        "content": "The forest was dark, and the wind howled through the trees."
    },
    {
        "speaker": "John",
        "content": "But what if we get lost? John asked, his voice trembling."
    },
    {
        "speaker": "Sarah",
        "content": "We'll be fine, Sarah reassured him. I've been here before."
    },
    {
        "speaker": "The forest",
        "content": "They continued walking, the sound of crunching leaves underfoot the only noise accompanying them."
    }
]
 > Text splitted to sentences.
["I don't think we should go any further, John said nervously."]
 > Processing time: 16.629746198654175
 > Real-time factor: 3.232877553959696
 > Text splitted to sentences.
["Come o

In [53]:
wav_files = [f"Results/output{i}.wav" for i in range(0, 5)]

combined_audio = AudioSegment.from_wav(wav_files[0])

# 逐步將剩下的音頻文件合併到第一個音頻文件中
for wav_file in wav_files[1:]:
    next_audio = AudioSegment.from_wav(wav_file)
    combined_audio += next_audio

# 將合併後的音頻文件保存
combined_audio.export("Results/combined_audio.wav", format="wav")

<_io.BufferedRandom name='Results/combined_audio.wav'>